In [16]:
pwd

'C:\\Users\\HOME\\Downloads'

In [50]:
import os

In [2]:
import sys
from scipy.io import loadmat
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

C:\Users\HOME\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [8]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [9]:
mat_file_list=['arrhythmia.mat', 'cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [30]:
data=loadmat('Dataset/data/cardio.mat')

In [31]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [32]:
data=loadmat('Dataset/data/arrhythmia.mat')

In [33]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue May 17 11:53:12 2016',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 75. ,   0. , 190. , ...,   2.9,  23.3,  49.4],
        [ 56. ,   1. , 165. , ...,   2.1,  20.4,  38.8],
        [ 54. ,   0. , 172. , ...,   3.4,  12.3,  49. ],
        ...,
        [ 36. ,   0. , 166. , ...,   1. , -44.2, -33.2],
        [ 32. ,   1. , 155. , ...,   2.4,  25. ,  46.6],
        [ 78. ,   1. , 160. , ...,   1.6,  21.3,  32.8]]),
 'y': array([[1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
 

In [41]:
df_columns=['Data','Samples','Dimensions','Outlier percentage','pca','mcd','ocsvm','lof','cblof','knn','hbos','abod','iforest','feature bagging']

In [42]:
roc_df=pd.DataFrame(columns=df_columns)

In [43]:
roc_df

,Data,Samples,Dimensions,Outlier percentage,pca,mcd,ocsvm,lof,cblof,knn,hbos,abod,iforest,feature bagging


In [44]:
prcn_df=pd.DataFrame(columns=df_columns)

In [45]:
prcn_df

,Data,Samples,Dimensions,Outlier percentage,pca,mcd,ocsvm,lof,cblof,knn,hbos,abod,iforest,feature bagging


In [46]:
time_df=pd.DataFrame(columns=df_columns)

In [47]:
time_df

,Data,Samples,Dimensions,Outlier percentage,pca,mcd,ocsvm,lof,cblof,knn,hbos,abod,iforest,feature bagging


In [66]:
from time import time
random_state=np.random.RandomState(42)

for mat_file in mat_file_list:
    print('/n....processing....',mat_file)
    mat=loadmat(os.path.join('Dataset/data',mat_file))
    X=mat['X']
    y=mat['y']
    outlier_fraction=np.count_nonzero(y)/len(y)
    outlier_percentage=round(outlier_fraction*100,ndigits=4)
    roc_list=[mat_file[:-4],X.shape[0],X.shape[1],outlier_percentage]
    prcn_list=[mat_file[:-4],X.shape[0],X.shape[1],outlier_percentage]
    time_list=[mat_file[:-4],X.shape[0],X.shape[1],outlier_percentage]
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=random_state)
    X_train_norm,X_test_norm=standardizer(X_train,X_test)
    classifiers = {'principle component analysis':PCA(contamination=outlier_fraction,random_state=random_state),'minimum covariance determinent':MCD(contamination=outlier_fraction,random_state=random_state),'one_class SVM':OCSVM(contamination=outlier_fraction),'local outlier fraction':LOF(contamination=outlier_fraction),'cluster_based lof':CBLOF(contamination=outlier_fraction,check_estimator=False,random_state=random_state),'k_nearest neighbour':KNN(contamination=outlier_fraction),'histogram based outlier score':HBOS(contamination=outlier_fraction),'angle_based outlier detection':ABOD(contamination=outlier_fraction),'i forest':IForest(contamination=outlier_fraction,random_state=random_state),'feature_bagging':FeatureBagging(contamination=outlier_fraction,random_state=random_state)}
    for clf_name,clf in classifiers.items():
        t0=time()
        clf.fit(X_train_norm)
        test=clf.decision_function(X_test_norm)
        t1=time()
        duration=t1-t0
        time_list.append(duration)
        roc=round(roc_auc_score(y_test,test),ndigits=4)
        prcn=round(precision_n_scores(y_test,test),ndigits=4)
        print('{clf_name}ROC:{roc} prcn{prcn} execution time:{duration}'.format(clf_name=clf_name,roc=roc,prcn=prcn,duration=duration))
        roc_list.append(roc)
        prcn_list.append(prcn)
    temp_df=pd.DataFrame(time_list).transpose()
    temp_df.columns= df_columns
    time_df=pd.concat([time_df,temp_df],axis=0)
        
    temp_df=pd.DataFrame(prcn_list).transpose()
    temp_df.columns= df_columns
    prcn_df=pd.concat([prcn_df,temp_df],axis=0)
        
    temp_df=pd.DataFrame(roc_list).transpose()
    temp_df.columns= df_columns
    roc_df=pd.concat([roc_df,temp_df],axis=0)
        

/n....processing.... arrhythmia.mat
principle component analysisROC:0.8 prcn0.55 execution time:0.26984357833862305


C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


minimum covariance determinentROC:0.8017 prcn0.5 execution time:4.900805711746216
one_class SVMROC:0.8017 prcn0.55 execution time:0.16993331909179688
local outlier fractionROC:0.7905 prcn0.5 execution time:0.31179237365722656
cluster_based lofROC:0.7806 prcn0.5 execution time:0.6346678733825684
k_nearest neighbourROC:0.781 prcn0.5 execution time:0.33580708503723145
histogram based outlier scoreROC:0.8108 prcn0.6 execution time:0.2718393802642822
angle_based outlier detectionROC:0.7616 prcn0.3 execution time:4.82062292098999
i forestROC:0.8211 prcn0.5 execution time:1.9019148349761963
feature_baggingROC:0.7858 prcn0.55 execution time:2.277726650238037
/n....processing.... cardio.mat
principle component analysisROC:0.9381 prcn0.5536 execution time:0.18942904472351074


C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-228.734080504027901 > -257.088833172604041). You may want to try with a higher value of support_fraction (current value: 0.509).
  RuntimeWarning)
C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-228.908420783967017 > -255.241163473552177). You may want to try with a higher value of support_fraction (current value: 0.509).
  RuntimeWarning)
C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarn

minimum covariance determinentROC:0.769 prcn0.4821 execution time:2.5691792964935303
one_class SVMROC:0.9194 prcn0.4643 execution time:0.3747577667236328
local outlier fractionROC:0.5389 prcn0.125 execution time:0.4297757148742676
cluster_based lofROC:0.7972 prcn0.5 execution time:0.6453995704650879
k_nearest neighbourROC:0.7065 prcn0.3036 execution time:0.5746781826019287
histogram based outlier scoreROC:0.8085 prcn0.3929 execution time:0.02998208999633789
angle_based outlier detectionROC:0.5569 prcn0.1964 execution time:1.6820294857025146
i forestROC:0.8971 prcn0.4464 execution time:1.7240159511566162
feature_baggingROC:0.5448 prcn0.1429 execution time:3.2551681995391846
/n....processing.... glass.mat
principle component analysisROC:0.4688 prcn0.0 execution time:0.030371665954589844


...

C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-47.455829387281156 > -74.432617965563637). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.249382438461424 > -78.483150432389763). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.219361792093167 > -79.158254313254176). You may want to try with a higher value of support_fraction (current value: 0.502).
  RuntimeWarning)
C:\User

C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.759481097442169 > -82.855462626059449). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.449802717934169 > -77.639524108233331). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.349033658619739 > -77.607335752930439). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\User

minimum covariance determinentROC:0.9896 prcn0.7442 execution time:48.049063205718994
one_class SVMROC:0.9917 prcn0.9533 execution time:237.1864297389984
local outlier fractionROC:0.5423 prcn0.1531 execution time:48.010093450546265
cluster_based lofROC:0.58 prcn0.2502 execution time:3.737832546234131
k_nearest neighbourROC:0.6522 prcn0.2306 execution time:34.161470890045166
histogram based outlier scoreROC:0.9845 prcn0.9386 execution time:0.06995177268981934
angle_based outlier detectionROC:0.6243 prcn0.2007 execution time:66.58795261383057
i forestROC:0.9966 prcn0.9458 execution time:13.478652238845825
feature_baggingROC:0.4858 prcn0.1046 execution time:204.44711470603943
/n....processing.... vertebral.mat
principle component analysisROC:0.3245 prcn0.0 execution time:0.008989810943603516


C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-23.158696735952276 > -41.065815815486225). You may want to try with a higher value of support_fraction (current value: 0.524).
  RuntimeWarning)
C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-23.236383616406748 > -41.292721878858138). You may want to try with a higher value of support_fraction (current value: 0.524).
  RuntimeWarning)
C:\Users\HOME\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-23.224109167983791 > -44.154594382010309). You may want to try with a higher value of support_fraction (current value: 0.524).
  RuntimeWarning)
C:\User

minimum covariance determinentROC:0.3298 prcn0.0 execution time:0.2728455066680908
one_class SVMROC:0.3668 prcn0.0 execution time:0.008911609649658203
local outlier fractionROC:0.3157 prcn0.0 execution time:0.012990713119506836
cluster_based lofROC:0.2892 prcn0.0 execution time:0.21687555313110352
k_nearest neighbourROC:0.2787 prcn0.0 execution time:0.03195905685424805
histogram based outlier scoreROC:0.2293 prcn0.0 execution time:0.00899505615234375
angle_based outlier detectionROC:0.2081 prcn0.0 execution time:0.21390271186828613
i forestROC:0.3139 prcn0.0 execution time:1.3282158374786377
feature_baggingROC:0.3298 prcn0.0 execution time:0.16092896461486816
/n....processing.... vowels.mat
principle component analysisROC:0.4703 prcn0.125 execution time:0.011832237243652344
minimum covariance determinentROC:0.6479 prcn0.0 execution time:3.899954080581665
one_class SVMROC:0.7111 prcn0.1875 execution time:0.18589258193969727
local outlier fractionROC:0.9345 prcn0.3125 execution time:0.19

In [67]:
roc_df

,Data,Samples,Dimensions,Outlier percentage,pca,mcd,ocsvm,lof,cblof,knn,hbos,abod,iforest,feature bagging
0,arrhythmia,452,274,14.6018,0.8,0.8017,0.8017,0.7905,0.7806,0.781,0.8108,0.7616,0.8211,0.7858
0,cardio,1831,21,9.6122,0.9381,0.769,0.9194,0.5389,0.7972,0.7065,0.8085,0.5569,0.8971,0.5448
0,glass,214,9,4.2056,0.4688,0.6875,0.4375,0.8281,0.7969,0.7031,0.6562,0.375,0.5,0.8125
0,ionosphere,351,33,35.8974,0.7658,0.9437,0.8089,0.8817,0.8656,0.9042,0.5658,0.8849,0.8374,0.8841
0,letter,1600,32,6.25,0.5695,0.8377,0.6412,0.8742,0.7749,0.8949,0.5933,0.8742,0.6189,0.8887
0,lympho,148,18,4.0541,1,1,1,1,1,1,1,0.8409,1,1
0,mnist,7603,100,9.2069,0.8486,0.8672,0.849,0.6862,0.8307,0.8436,0.5609,0.7609,0.7755,0.6879
0,musk,3062,166,3.1679,0.9998,0.9989,1,0.6345,1,0.8021,0.9998,0.1551,1,0.6215
0,optdigits,5216,64,2.8758,0.5201,0.3981,0.5277,0.5178,0.804,0.3813,0.8425,0.5169,0.7337,0.5136
0,pendigits,6870,16,2.2707,0.929,0.8267,0.9292,0.4983,0.972,0.7484,0.9381,0.6813,0.9659,0.4903


In [68]:
prcn_df

,Data,Samples,Dimensions,Outlier percentage,pca,mcd,ocsvm,lof,cblof,knn,hbos,abod,iforest,feature bagging
0,arrhythmia,452,274,14.6018,0.55,0.5,0.55,0.5,0.5,0.5,0.6,0.3,0.5,0.55
0,cardio,1831,21,9.6122,0.5536,0.4821,0.4643,0.125,0.5,0.3036,0.3929,0.1964,0.4464,0.1429
0,glass,214,9,4.2056,0,0,0,0,0,0,0,0,0,0
0,ionosphere,351,33,35.8974,0.5714,0.8571,0.6571,0.6571,0.6857,0.8571,0.2571,0.8,0.6286,0.6857
0,letter,1600,32,6.25,0.1944,0.3056,0.1944,0.4167,0.2778,0.4444,0.2222,0.5,0.1667,0.4167
0,lympho,148,18,4.0541,1,1,1,1,1,1,1,0,1,1
0,mnist,7603,100,9.2069,0.3803,0.3521,0.385,0.3192,0.385,0.4178,0.0845,0.3474,0.2535,0.3146
0,musk,3062,166,3.1679,0.9231,0.9231,1,0.1923,1,0.1538,0.9615,0.0385,1,0.1923
0,optdigits,5216,64,2.8758,0,0,0,0.0222,0,0,0.2222,0.0222,0.0222,0.0222
0,pendigits,6870,16,2.2707,0.25,0.0625,0.2708,0.0833,0.3125,0.125,0.2708,0.1042,0.2708,0.0833


In [69]:
time_df

,Data,Samples,Dimensions,Outlier percentage,pca,mcd,ocsvm,lof,cblof,knn,hbos,abod,iforest,feature bagging
0,arrhythmia,452,274,14.6018,0.269844,4.90081,0.169933,0.311792,0.634668,0.335807,0.271839,4.82062,1.90191,2.27773
0,cardio,1831,21,9.6122,0.189429,2.56918,0.374758,0.429776,0.6454,0.574678,0.0299821,1.68203,1.72402,3.25517
0,glass,214,9,4.2056,0.0303717,0.306859,0.0119925,0.0179839,0.261856,0.0280056,0.0109942,0.150883,1.54415,0.161916
0,ionosphere,351,33,35.8974,0.0604448,0.343657,0.0259838,0.0400066,0.243831,0.0739646,0.0429747,0.316842,1.52214,0.381776
0,letter,1600,32,6.25,0.0219634,5.24967,0.346831,0.5207,0.563693,0.684606,0.049974,1.86093,2.04184,4.22161
0,lympho,148,18,4.0541,0.0149975,0.211904,0.00599718,0.0100172,0.220874,0.0229876,0.0179901,0.10494,1.49914,0.121929
0,mnist,7603,100,9.2069,0.496918,16.1103,20.6042,26.2993,4.04469,25.0486,0.219875,30.6875,7.89451,185.841
0,musk,3062,166,3.1679,0.545083,65.5834,4.71427,8.80796,1.62307,8.68204,0.262848,10.3511,5.10106,64.1563
0,optdigits,5216,64,2.8758,0.201886,4.81771,5.99257,6.63421,1.94768,7.13789,0.154913,12.5778,3.79883,53.9412
0,pendigits,6870,16,2.2707,0.0340123,9.38833,4.41447,2.34866,1.12138,2.23269,0.0359857,6.58723,2.80541,16.9643
